# ReOpt

- API structure: https://developer.nrel.gov/api/reopt/stable/help?API_KEY=DEMO_KEY
- Models.py: https://github.com/NREL/REopt_API/blob/master/reoptjl/models.py

In [1]:
import pandas as pd
import os, sys
from src.post_and_poll import get_api_results
from utils import import_series, import_df, plotly_stacked, parse_dispatch_series

# to silence warnigns
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

pd.options.plotting.backend='plotly'

API_KEY = '0bgdMrMpcFEfzIuYLRFpEfPeUqabOQKo5RagyRPV' 

Create POST
- To turn on a technolgoy just add a key like `"ElectricStorage":{}`

In [26]:
def run_reopt(array,solar_kw=1000,batt_kw=250,batt_h=2,print_results=False):

    post = {
        "Site": {
            "longitude": 45.505890265478996, 
            "latitude": 9.160999024421262
        },
        "PV": {
            "production_factor_series":import_df(f'data/pasadena_2019_1h_367mods_s20w90.csv',col=array,scale=128),
            "array_type": 0,
            "min_kw":solar_kw,
            "max_kw":solar_kw,
            "can_net_meter":True,
            "installed_cost_per_kw":2.50,
        },
        "ElectricLoad": {
            'loads_kw':import_df('data/acn_caltech_profile_1y.csv','Load (kW)',resamp='1h')
            #"loads_kw":import_df('data/load/2019_JPL_v4.csv','Power_kW',resamp='1h'),
        },
        "ElectricStorage": {
            "min_kw":batt_kw,
            "max_kw":batt_kw,
            "min_kwh":batt_kw*batt_h,#140
            "max_kwh":batt_kw*batt_h,#140
            "can_grid_charge":True,
            "installed_cost_per_kw":0, #455
            "installed_cost_per_kwh":0, #910
            "soc_min_fraction":0.0,
            "soc_max_fraction":1.0,
            "soc_init_fraction":0.0,
        },

        "ElectricTariff": {
            #'tou_energy_rates_per_kwh':(10*[.20]+8*[0.30]+6*[0.20])*365,
            #'wholesale_rate':[0.0]*8760,
            'wholesale_rate':import_df('data/PG&E NBT EEC Values 2024 Vintage.csv','Price',resamp='1h'),
            'urdb_label': '5c817fcf7bece2299eceb39b', # General Service Time-of-use Electric Vehicle Charging - Demand Metered:TOU-EV-4 (50kV - 220 kV) - Closed Rate
            #'wholesale_rate':[0.05]*8760,
            #'urdb_label': '652f274fcb43e728cc0155cf',
            #"tou_energy_rates_per_kwh":import_series('data/tariff/jplv4_price-buy_aut19_35040.csv'),
            #"wholesale_rate":import_series('data/tariff/jplv4_price-sell_aut19_35040.csv'),
            #'coincident_peak_load_active_time_steps':(16*[0]+4*[1]+4*[0])*365,
            #'coincident_peak_load_charge_per_kw': 30
        },
        "ElectricUtility":{
            "net_metering_limit_kw":0,
            
        },
        "Financial": {
            "elec_cost_escalation_rate_fraction": 0.026,
            "owner_discount_rate_fraction": 0.081,
            "analysis_years": 1,
            "offtaker_tax_rate_fraction": 0.4,
            "owner_tax_rate_fraction": 0.4,
            "om_cost_escalation_rate_fraction": 0.025
        }
    }

    outputs_file_name = "results_file"
    root_url = "https://developer.nrel.gov/api/reopt/stable" # /stable == /v3 
    
    try:
        api_response = get_api_results(post=post, 
                                    API_KEY=API_KEY, 
                                    api_url=root_url, 
                                    results_file= f'outputs/{outputs_file_name}.json', 
                                    run_id=None)
    except:
        print('API request failed')
        
    if api_response is not None:
        cost =      api_response["outputs"]["ElectricTariff"]["year_one_energy_cost_before_tax"]
        revenue =   api_response["outputs"]["ElectricTariff"]["year_one_export_benefit_before_tax"]
        netcost = cost - revenue

        if print_results:    
            print('Status = ',              api_response["status"])
            print("Energy cost ($) = ",     cost)
            print('Energy revenue ($) = ',  revenue)
            print('Net cost ($) = ',        netcost)
            print('PV Size (kW) = ',        api_response["outputs"]["PV"]["size_kw"])
            if "ElectricStorage" in api_response["outputs"].keys():
                print('Storage Size (kW-kwh) = ',api_response["outputs"]["ElectricStorage"]["size_kw"],'-',api_response["outputs"]["ElectricStorage"]["size_kwh"])
    else:
        print('API request failed')
        cost = pd.NA
        revenue = pd.NA
        netcost = pd.NA
    return api_response, cost, netcost

def compare_to_s20(arrays:list,solar_kw:int,batt_kw:int,batt_h:int,print_results=False):
    cost_red,cost_red_pct,netcost_red = [], [], []
    resp, cost_base, netcost_base = run_reopt('0w90',solar_kw,batt_kw,batt_h,print_results)
    for array in arrays:
        resp, cost, netcost = run_reopt(array,solar_kw,batt_kw,batt_h,print_results)
        cost_red.append(        cost_base - cost)
        cost_red_pct.append(    1 - cost/abs(cost_base))
        netcost_red.append(     netcost_base - netcost)
        if print_results:
            print(f'\nCost Reduction {array} ($) = {cost_base-cost:.2f} ({100*(1-cost/abs(cost_base)):.1f}%)')
            print(f'NetCost Reduction {array} ($) = {netcost_base-netcost:.2f} ({100*(1-netcost/abs(netcost_base)):.1f}%)\n')
    return resp, cost_base, cost_red, cost_red_pct, netcost_red

In [ ]:
#run_reopt('0w90',125,25,2)

In [28]:
arrays = ['25w90','50w90','75w90','100w90']

cols = ['SolarKW','BattKW','BattH','BaseCost']
df = pd.DataFrame([],columns=cols+['Reduc_'+x for x in arrays]+['Reduc%_'+x for x in arrays]+['NetReduc_'+x for x in arrays])

for solar_kw in [100]:#,125,150,175,200]:
    for batt_kw in [25]:#,50,75,100]:
        for batt_h in [1]:#,2,3]:
            resp, basecost, cost_red, cost_red_pct, netcost_red = compare_to_s20(arrays,solar_kw,batt_kw,batt_h,print_results=False)
            df.loc[len(df)] = [solar_kw,batt_kw,batt_h,basecost]+cost_red+cost_red_pct+netcost_red
            df.to_csv('outputs/results.csv',index=False)
            print(df.iloc[:,:7])

   SolarKW  BattKW  BattH  BaseCost  Reduc_25w90  Reduc_50w90  Reduc_75w90
0    100.0    25.0    1.0   5623.86      -307.15       -17.41      -307.15


# Plot

In [34]:
df = parse_dispatch_series(resp)

f = plotly_stacked(df,solar='pv',load='load',batt='batt',utility='grid',soc='soc',
                   plot_theme='plotly')#,ylim=[0,50])